In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import time
import glob
import sys
import os
import re

In [2]:
os.chdir("..")
from src.utils.general import busca_tipo_tesis,extraccion_encabezado,busqueda_ponentes,busqueda_fecha,busqueda_secretarios,crea_tabla
from src.utils.limpieza import limpieza_texto
os.chdir("notebooks")

In [3]:
path, dirs, files = next(os.walk("../data_prueba"))
file_count = len(files)
print(file_count)

10001


In [17]:
ruta_txt = '../data_prueba/*.txt'

In [19]:
tabla_tesis_jurisprudencia = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia', '#_ponente', 'ponente', '#_secretario', 'secretario', 'fecha'])

t0 = time.time()
n = len(glob.glob(ruta_txt))
n_juris_reiter_prueba = 0

for i in range(n):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_prueba += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia = tabla_tesis_jurisprudencia.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_prueba, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_prueba)


Van 1000 iteraciones de 10000 en 22.96 segundos
Hay 187 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 10000 en 50.01 segundos
Hay 313 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 10000 en 79.38 segundos
Hay 412 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 10000 en 110.21 segundos
Hay 546 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 10000 en 141.81 segundos
Hay 665 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 10000 en 174.55 segundos
Hay 787 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 10000 en 205.83 segundos
Hay 902 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 10000 en 236.52 segundos
Hay 1013 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 10000 en 268.09 segundos


In [20]:
tabla_tesis_jurisprudencia.head(20)

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
0,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,1,Jaime Raúl Oropeza García,1,Roberto Carlos Moreno Zamorano,19 de febrero de 2004
1,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,2,Ma. del Pilar Núñez González,2,José Fernández Martínez,17 de febrero de 2005
2,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,3,Manuel Rojas Fonseca,3,Juan Carlos Ríos López,23 de febrero de 2006
3,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,4,Jaime Raúl Oropeza García,4,Alejandro Ramos García,25 de mayo de 2006
4,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Décima Época,Administrativa,5,Jaime Raúl Oropeza García,5,Clemente Delgado Salgado,30 de noviembre de 2006
0,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",1,Olga Sánchez Cordero de García Villegas,1,Ana Carolina Cienfuegos Posada,26 de octubre de 2005
1,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",2,Olga Sánchez Cordero de García Villegas,2,Constanza Tort San Román,30 de noviembre de 2005
2,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",3,Olga Sánchez Cordero de García Villegas,3,Francisco Octavio Escudero Contreras,2 de julio de 2008
3,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",4,Olga Sánchez Cordero de García Villegas,4,Francisco Octavio Escudero Contreras,3 de diciembre de 2008
4,159814,1a./J. 71/2014 (9a.),Primera Sala,Décima Época,"Constitucional, Penal",5,José de Jesús Gudiño Pelayo,5,Carmina Cortés Rodríguez,1 de septiembre de 2010


In [21]:
tabla_tesis_jurisprudencia.describe()

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
count,6410,6410,6410,6410,6410,6410,6410,6410,6410,6410
unique,1282,1277,4,2,28,5,537,5,1490,1486
top,159807,III.1o.T.Aux. J/1,Tribunales Colegiados de Circuito,Novena Época,Administrativa,1,José Ramón Cossío Díaz,1,"Georgina Laso de la Vega Romero, Sofía Verónic...",19 de junio de 2008
freq,5,10,3955,5405,1300,1282,413,1282,449,460


In [22]:
tabla_tesis_jurisprudencia.ponente.value_counts()

José Ramón Cossío Díaz                     413
Margarita Beatriz Luna Ramos               311
Sergio A. Valls Hernández                  266
Olga Sánchez Cordero de García Villegas    266
Juan N. Silva Meza                         222
                                          ... 
Humberto Castañeda Martínez                  1
Lorena Ortuño Yáñez                          1
Ana Gabriela Urbina Roca                     1
Alfonso M. Cruz Sánchez                      1
Ángel Michel Sánchez                         1
Name: ponente, Length: 537, dtype: int64

In [23]:
tabla_tesis_jurisprudencia.secretario.value_counts()

Georgina Laso de la Vega Romero, Sofía Verónica Ávalos Díaz, María Marcela Ramírez Cerrillo, Carmen Vergara López, Gustavo Ruiz Padilla y Luciano Valadez Pérez    449
María Estela Ferrer Mac Gregor Poisot, Ricardo Manuel Martínez Estrada, Fanuel Martínez López, Jorge Luis Revilla de la Torre y Juan Carlos Roa Jacobo             147
María Constanza Tort San Román, Gustavo Ruiz Padilla, Israel Flores Rodríguez y Fernando Tinoco Ortiz                                                              115
Rolando Javier García Martínez                                                                                                                                      75
Alfredo A. Martínez Jiménez                                                                                                                                         65
                                                                                                                                                                  ...

In [24]:
tabla_tesis_jurisprudencia.fecha.value_counts()

19 de junio de 2008         460
4 de noviembre de 2009      177
28 de noviembre de 2007     109
22 de septiembre de 2010     67
15 de marzo de 2011          60
                           ... 
15 de mayo de 2007            1
5 de octubre de 2010          1
23 de abril de 2010           1
25 de agosto de 2005          1
21 de mayo de 2007            1
Name: fecha, Length: 1486, dtype: int64

In [25]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia).to_csv('tesis_juris_reiter_setprueba.csv', index=False, encoding='utf-8-sig')

### Debug individual por tesis

In [7]:
def funcion_auxiliar_ponentes(x0,x1,x2,x3):
    if len(x0)!= 0 :
        result = x0
    elif len(x1)!= 0:
        result = x1
    elif len(x2)!= 0:
        result =["No se menciona el nombre del ponente sólo del relator"]
    elif x3 == 1:
        result =["Revisar texto tesis"]    
    return result


def funcion_auxiliar_secretarios(x0,x1,x10): 
    if len(x0)!= 0 :
        result = x0
    elif len(x1)!= 0:
        result = ["NA"]
    elif x10 == 1:
        result =['NA']    
    return result

In [21]:
tesis_a_revisar = '232480'

In [22]:
dir_text = "../data_04/Tesis"+tesis_a_revisar+".txt"

file = open(dir_text,'r', encoding='utf8')
text = file.read()
file.close()
    
#Eliminar tabulaciones adicionales
text_aux = text.strip()
    
#Posicion en el texto de las variables
pos_registro_digital = text_aux.find('Registro digital:')
pos_final_registro_digital = text_aux[pos_registro_digital:].find('\n')

pos_instancia = text_aux.find('Instancia:')
pos_final_instancia_aux1 = text_aux[pos_instancia:].find('\n\n')

lista  = []

lista.append(text_aux[pos_instancia:].find('Primera Época'))
lista.append(text_aux[pos_instancia:].find('Segunda Época'))
lista.append(text_aux[pos_instancia:].find('Tercera Época'))
lista.append(text_aux[pos_instancia:].find('Cuarta Época'))
lista.append(text_aux[pos_instancia:].find('Quinta Época'))
lista.append(text_aux[pos_instancia:].find('Sexta Época'))
lista.append(text_aux[pos_instancia:].find('Séptima Época'))
lista.append(text_aux[pos_instancia:].find('Octava Época'))
lista.append(text_aux[pos_instancia:].find('Novena Época'))
lista.append(text_aux[pos_instancia:].find('Décima Época'))
lista.append(text_aux[pos_instancia:].find('Undécima Época'))

pos_final_instancia_aux2 = min([x for x in lista if x > -1])
    
if pos_final_instancia_aux1 < pos_final_instancia_aux2:
    pos_final_instancia = pos_final_instancia_aux1
else:
    pos_final_instancia = pos_final_instancia_aux2

pos_epoca_aux1 = text_aux.find('\n\n')

lista  = []

lista.append(text_aux.find('Primera Época'))
lista.append(text_aux.find('Segunda Época'))
lista.append(text_aux.find('Tercera Época'))
lista.append(text_aux.find('Cuarta Época'))
lista.append(text_aux.find('Quinta Época'))
lista.append(text_aux.find('Sexta Época'))
lista.append(text_aux.find('Séptima Época'))
lista.append(text_aux.find('Octava Época'))
lista.append(text_aux.find('Novena Época'))
lista.append(text_aux.find('Décima Época'))
lista.append(text_aux.find('Undécima Época'))

pos_epoca_aux2 = min([x for x in lista if x > -1])

if pos_epoca_aux1 < pos_epoca_aux2:
    pos_epoca = pos_epoca_aux1
else:
    pos_epoca = pos_epoca_aux2

if pos_epoca_aux1 < pos_epoca_aux2:
    pos_final_epoca = text_aux[pos_epoca+2:].find('\n\n')
else:
    pos_final_epoca = text_aux[pos_epoca+2:].find('\n')
        
pos_materia = text_aux.find('Materia(s):')
pos_final_materia = text_aux[pos_materia:].find('\n\n')

pos_tesis = text_aux.find('Tesis:')
pos_final_tesis = text_aux[pos_tesis:].find('\n\n')

#pos_fuente = text_aux.find('Fuente:')
#pos_final_fuente = text_aux[pos_fuente:].find('.')

pos_tipo = text_aux.find('Tipo:')
if text_aux[pos_tipo:].find('\n\n') < text_aux[pos_tipo:].find('\n') :
    pos_final_tipo = text_aux[pos_tipo:].find('\n\n')
else:
    pos_final_tipo = text_aux[pos_tipo:].find('\n')-2

pos_encabezado = text_aux[pos_tipo:].find('\n\n')

#Tamano de las variables
len_registro_digital = len('Registro digital:')
len_instancia = len('Instancia:')
if pos_epoca_aux1 < pos_epoca_aux2:
    len_epoca = len('\n\n')
else:
    len_epoca = 0
len_materia = len('Materia(s):')
len_tesis = len('Tesis:')
#len_fuente = len('Fuente:')
len_tipo = len('Tipo:')
    
tipo = text_aux[pos_tipo+len_tipo:pos_tipo+pos_final_tipo+2].replace('\n',' ').strip()
tesis = text_aux[pos_tesis+len_tesis:pos_tesis+pos_final_tesis+2].replace('\n',' ').strip()
instancia = text_aux[pos_instancia+len_instancia:pos_instancia+pos_final_instancia].replace('\n',' ').strip()
epoca = text_aux[pos_epoca+len_epoca:pos_epoca+pos_final_epoca+2].replace('\n',' ').strip()
materia = text_aux[pos_materia+len_materia:pos_materia+pos_final_materia+2].replace('\n',' ').strip()

if tipo == "Jurisprudencia":
        
    if len(re.findall('Ponente',text_aux)) >= 5:
        
        #Búsqueda de Id
        encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
        id_tesis = [encontrar_id_tesis.group(1)]

        #Limpieza Texto
        tesis_borrar = ''.join(id_tesis)
        text_aux = text_aux.strip()
        text_aux = text_aux.replace('\n\n','###')
        text_aux = text_aux.replace('\n',' ')
        text_aux = text_aux.replace('Ley de Amparo','ley_de_amparo')
        text_aux = text_aux.replace('Competencia Económica','competencia_económica')
        text_aux = text_aux.replace('Fomento a la Competencia','Fomento a la competencia')
        text_aux = text_aux.replace('del año','de')
        text_aux = text_aux.replace('Presidente Ponente','Presidente-ponente')
        text_aux = text_aux.replace('autorizados por el Secretario','autorizados por el secr.')
        text_aux = text_aux.replace('Ponente: La publicación no menciona el nombre del ponente.','La publicación no menciona el nombre del ponente.')
        text_aux = text_aux.replace(' en sesión de 5 de marzo de 1996','')
        text_aux = text_aux.replace('29 se septiembre de 1993','29 de septiembre de 1993')
        text_aux = text_aux.replace('12 se agosto de 1976','12 de agosto de 1976')
        text_aux = text_aux.replace('10 de 1988','10 de FALTAMES de 1988')
        text_aux = text_aux.replace('* de abril de 1969','99 de abril de 1969')
        text_aux = text_aux.replace('7 de agosto de 1984. 7 de agosto de 1984.','7 de agosto de 1984.')
        text_aux = text_aux.replace('4 de julio de 975','4 de julio de 1975')
        text_aux = text_aux.replace(r'1o.','01')
        text_aux = text_aux.replace(r'1°','01')
        text_aux = text_aux.replace(r'1º','01')
        text_aux = text_aux.replace('Pág. 1 de 1 ','')
        text_aux = text_aux.replace('Pág. 1 de 2 ','')
        text_aux = text_aux.replace('Pág. 2 de 2 ','')
        text_aux = text_aux.replace('Pág. 1 de 3 ','')
        text_aux = text_aux.replace('Pág. 2 de 3 ','')
        text_aux = text_aux.replace('Pág. 3 de 3 ','')
        text_aux = text_aux.replace('Pág. 1 de 4 ','')
        text_aux = text_aux.replace('Pág. 2 de 4 ','')
        text_aux = text_aux.replace('Pág. 3 de 4 ','')
        text_aux = text_aux.replace('Pág. 4 de 4 ','')
        text_aux = text_aux.replace('Pág. 1 de 5 ','')
        text_aux = text_aux.replace('Pág. 2 de 5 ','')
        text_aux = text_aux.replace('Pág. 3 de 5 ','')
        text_aux = text_aux.replace('Pág. 4 de 5 ','')
        text_aux = text_aux.replace('Pág. 5 de 5 ','')
        text_aux = text_aux.replace('Pág. 1 de 6 ','')
        text_aux = text_aux.replace('Pág. 2 de 6 ','')
        text_aux = text_aux.replace('Pág. 3 de 6 ','')
        text_aux = text_aux.replace('Pág. 4 de 6 ','')
        text_aux = text_aux.replace('Pág. 5 de 6 ','')
        text_aux = text_aux.replace('Pág. 6 de 6 ','')
        text_aux = text_aux.replace('http://sjf2.scjn.gob.mx/detalle/tesis/','')
        text_aux = text_aux.replace(tesis_borrar,'')
        text_aux = text_aux.replace('  Fecha de impresión','')
        text_aux = text_aux.replace('Fecha de impresión','')
        text_aux = text_aux.replace(' 06/07/2021','')
        text_aux = text_aux.replace(' 07/07/2021','')
        text_aux = text_aux.replace(' 08/07/2021','')
        text_aux = text_aux.replace(' 09/07/2021','')
        text_aux = text_aux.replace(' 10/07/2021','')
        text_aux = text_aux.replace('  \x0c','')
        text_aux = text_aux.replace('######\x0c','')
        text_aux = text_aux.replace('Semanario Judicial de la Federación ','')
        text_aux = re.sub(r'Se formó jurisprudencia aunque no.*',"",text_aux)
        text_aux = text_aux.replace(r'(DA. 903/88. Impulsora Industrial de Ingeniería, S. A. de C. V. 15 de junio de 1988. Unanimidad de votos. Ponente: Genaro David Góngora Pimentel. Secretaria: Adriana Leticia Campuzano Gallegos)','')
        #text_aux = text_aux.replace('######','###')

        #Búsqueda de párrafo donde se mencione ponente con fecha
        #ponente_fecha_aux = re.findall(r'(?:Amparo|Revisión|Contradicción|Queja|Facultad|Incidente|Reclamación|Inconformidad|Competencia|Recurso de|Impedimento|Acción de inconstitucionalidad)([\s\S]*?)(Ponente: |Ponente )([\s\S]*?)(Secretari\w: |Secretari\ws:|Secretari\w. )([\s\S]*?)(?:\.?)', text_aux)
        ponente_fecha_aux = re.findall(r'(?:Amparo|Revisión|Contradicción|Queja|Facultad|Incidente|Reclamación|Inconformidad|Competencia|Recurso de|Impedimento|Acción de inconstitucionalidad|Varios |Controversia constitucional|Reconocimiento de inocencia|Improcedencia|Aclaración de sentencia en la contra|Repetición|Inejecuci|Expediente vario|Solicitud|Acumulaci\wn|Trámite|Consulta|Recurso en|Juicio|Reposici\wn|Incompetencia|Incidente|Vol\wmen \d{1,2}, p\wgina|Indulto necesario)([\s\S]*?)(Ponente: |Ponente |Ponente\.|Ponente:\.|La publicación no menciona el nombre del ponente)([\s\S]*?)(?:Secretari\w: |Secretari\ws:|Secretari\w. )?([\s\S]*?)(?:\.?)', text_aux)
        ponente_fecha_texto = ''.join([''.join(j) for j in ponente_fecha_aux])
        
        #Búsqueda de Ponente
        #x0_ponente = re.findall("Ponente:([\s\S]*?)(?:.###|. Secretari|.Secretari|,|.\nSecretari|;|. Nota:|Pág.|Vol|.Tom|. Véase|. Amparo|. Séptima Época|. Sexta Época)",ponente_fecha_texto) 
        x0_ponente = re.findall("(?:Ponente: Secretario Lic\. |Ponente: |Ponente |Ponente. |Ponentes: |Ponente:\.|Ponente:)([\s\S]*?)(?:.###|. Secretar|.Secretar|,|.\nSecretar|\. secretari|\. Relacionada|\. Disidente|\. Improcedencia|\. Semanario|\. Encargad|\. Competencia|\. Incidente|\. Queja|\. Reclamaci|\. Recurso|\. Juicio|\. Jurisprudencia| Magistrado|\. Tomo|\. Vol\wmen|;|\. Nota|\. En| en funciones| en sustitución| y el engrose|Pág\.|Vol|\.Tom|\. V\wase|\. V\wanse|\. Revisión|\. Amparo|\. Ausente|\. Séptima|\. Sexta|\.\"\.)",text_aux) 
        x1_ponente = re.findall("publicación ([\s\S]*?) del ponente",text_aux )
        x2_ponente = re.findall("Relator: ([\s\S]*?)\.",text_aux ) #re.findall(r"Relator",text_aux)
        x3_ponente = 1
        ponentes = funcion_auxiliar_ponentes(x0_ponente,x1_ponente,x2_ponente,x3_ponente)
        
        if len(re.findall('La publicación no menciona el nombre del ponente',text_aux)) != 0: 
            NA_ponentes = [('NA') for i in range(len(re.findall('La publicación no menciona el nombre del ponente',text_aux)))]
            ponentes.extend(NA_ponentes)
        if len(x2_ponente) != 0: 
            ponentes.extend(x2_ponente)

        #Búsqueda de Fecha
        #fecha_ponente = re.findall(r'\d{1,2} de [a-z]*? de \d{4}', ponente_fecha_texto)
        #fecha = re.findall('(\d{1,2} de [a-z]*?( de)*? \d{4})',ponente_fecha_texto)
        fecha = re.findall('(\d{1,2}( de|de)*? [a-zA-Z]{4,10}( de| del)*? \d{4})',ponente_fecha_texto)
        fecha_aux = pd.DataFrame(fecha,columns=['fecha','del1','del2']).drop(columns=['del1','del2'])
        fecha = fecha_aux['fecha'].values.tolist()
        if len(re.findall('La publicación no menciona la fecha de resolución',text_aux)) != 0: 
            NA_fecha = [('NA') for i in range(len(re.findall('La publicación no menciona la fecha de resolución',text_aux)))]
            fecha.extend(NA_fecha)
                
        #Búsqueda de Secretario
        #x0_secretario = re.findall("(?:Secretari\w: |Secretari\ws: |Secretari\w. )([\s\S]*?)(\.| Amparo)",text_aux)
        #x0_aux = pd.DataFrame(x0_secretario,columns=['secretario','delete']).drop(columns=['delete'])
        #x0_secretario = x0_aux['secretario'].values.tolist()
        #x1_secretario = re.findall("(?:La publicación |La publicacion)([\s\S]*?) (?:del ponente|el ponente)",text_aux)
        ##x10_secretario = 10
        ##secretarios = funcion_auxiliar_secretarios(x0_secretario,x1_secretario) #,x10_secretario
        #secretarios = x0_secretario
        
        x0_secretario = re.findall("(?:Secretar\w\w: |Secretari\ws: |Secretari\w\. |secretari\w: )([\s\S]*?)(?:\.###|\. Amparo|\. Facultad| Amparo directo|\. Inconformidad|\. Acción|\. Varios|\. Controversia|\. Reconocimiento|\. Expediente vario|\. Consulta|\. Reposici\wn|\. \(|\. La |\. Incompetencia|\. Vol\wmenes|se encargo|Quinta Epoca:|\. Hay cosa|\. Acumulación|Impedimento|Antecedente:|Revisión|Expediente|Los señores|Jurisprudencia|demás que las|XIV.|las demás que|fallado|no superan|Aplicada|De conformidad|contra el|para el|Lo resolvió|Secretarrio:|Presidente|Secretario encargado|ha estimado|Disidente.|Veáse|\. Impedido:|Vease:|Por |Sexta Epoca|Integró|Para|Tribunal|\. Ausente|votos|_________________|Texto|Secretarios|Reproduce|Véanse:|Ponente:|EL|Tomo|Resuelto|Ausentes|Que |Varios|Con |\. Aclaración|Tomando|\. Juicio|Relator|\. Ministro|Incidente|/.|Texto aprobado|Relacionada|Séptima|Acción|Facultad|Controversia|\. Solicitud|Sostiene|La |\. Inejecución|Octava|En |Este |Se |\. Repetición|Reitera|\. El |Contradicción|La presente|El Tribunal|Tesis|La Primera|Sostienen|Informe|Véase|Esta|Importa|Disidentes:|Voto|Ausente:|Secretario:|###|Inconformidad|\. Conflicto|Nota|Criterios|Competencia|Precedente|\. Recurso| Recurso de|Secretaria|;|\. Nota\:|\. Notas\:|Nota\:|Pág\.|Vol|\. Volumen|\. Volúme|\.Tom|\. Véase|\. \. Amparo|\. Amparo di|\.   Amparo dir|\. Amparo in|Amparo en|\. Undécima Época|\. Décima Época|\. Novena Época|\. Octava Época|\. Séptima Época|\. Sexta Época|\. Quinta Época|\. Cuarta Época|\. Tercera Época|\. Segunda Época|\. Primera Época|\. Undécima Epoca|\. Décima Epoca|\. Novena Epoca|\. Octava Epoca|\. Séptima Epoca|\. Sexta Epoca|\. Quinta Epoca|\. Cuarta Epoca|\. Tercera Epoca|\. Segunda Epoca|\. Primera Epoca|\. Improcedencia civil|Sostiene la misma tesis:|\. Revisión fiscal|\. Unanimidad|\. Queja|Queja|\. Competencia|\. Reclamación|\.   Reclamación /.|\.  de enero de|\.  de febrero de|\.  de marzo de|\.  de abril de|\.  de mayo de|\.  de junio de|\.  de julio de|\.  de agosto de|\.  de septiembre de|\.  de octubre de|\.  de noviembre de|\.  de diciembre de|\. Disidente:|\. Precedentes|\. Improcedencia|\. Impedimento|\. Excusa|\. Incidente|\. Encargad|\. Revisión|\. Tomo|por licencia concedida|en funciones de Magistrado por Ministerio de Ley|\. Engrose: |\. Criterios|\. Tesis|\.   Tesis|\. El Tribunal|\. Contradicción de tesis|\. Texto|\. NOTA|\. Recurso de|\.   El Tribunal|\. Véanse|\.   Revisión|.   Notas|\. Semanario|\. Secretario)", text_aux)                                
        x1_secretario = re.findall("(?:La publicación |La publicacion)([\s\S]*?) (?:del ponente|el ponente)",text_aux)
        x10_secretario = 1
        secretarios = funcion_auxiliar_secretarios(x0_secretario,x1_secretario,x10_secretario)
        if len(secretarios) != len(ponentes):
            NA_sec = [('NA') for i in range(len(ponentes)-len(secretarios))]
            secretarios.extend(NA_sec)
    
        #impresión debug
        print(id_tesis)
        print(fecha)
        print('Ponentes:',ponentes)
        print('Secretarios:',secretarios)

        #Vectores para crear tabla
        vector_id_tesis = len(ponentes)*id_tesis
        vector_numero = [(i+1) for i in range(len(ponentes))]
        vector_tesis = [(tesis) for i in range(len(ponentes))]
        vector_instancia = [(instancia) for i in range(len(ponentes))]
        vector_epoca = [(epoca) for i in range(len(ponentes))]
        vector_materia = [(materia) for i in range(len(ponentes))]

        tabla_iteracion = pd.DataFrame({'id_tesis':vector_id_tesis,'tesis':vector_tesis,'instancia':vector_instancia,'epoca':vector_epoca,'materia':vector_materia,'#_ponente':vector_numero,'ponente':ponentes,'#_secretario':vector_numero, 'secretario':secretarios,'fecha':fecha})
        #tabla_tesis_jurisprudencia = tabla_tesis_jurisprudencia.append(tabla_iteracion)
        
        print(instancia)
        print(epoca)
        print(materia)
        print(tesis)

['232480']
['11 de junio de 1968', '5 de marzo de 1977', '15 de marzo de 1977', '22 de noviembre de 1977', '18 de marzo de 1980', '8 de mayo de 1979', '18 de noviembre de 1980', '17 de marzo de 1981', '17 de marzo de 1981', '16 de febrero de 1982']
Ponentes: ['Ernesto Solís López', 'Raúl Cuevas Mantecón', 'J. Ramón Palacios Vargas', 'Raúl Cuevas Mantecón', 'Arturo Serrano Robles', 'Mario G. Rebolledo', 'Atanasio González Martínez', 'Fernando Castellanos Tena', 'Atanasio González Martínez', 'NA']
Secretarios: ['NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA']
Pleno
Séptima Época
Constitucional, Común



In [18]:
tabla_iteracion

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
0,232480,,Pleno,Séptima Época,"Constitucional, Común",1,Ernesto Solís López,1,NA,11 de junio de 1968
1,232480,,Pleno,Séptima Época,"Constitucional, Común",2,Raúl Cuevas Mantecón,2,NA,5 de marzo de 1977
2,232480,,Pleno,Séptima Época,"Constitucional, Común",3,J. Ramón Palacios Vargas,3,NA,15 de marzo de 1977
3,232480,,Pleno,Séptima Época,"Constitucional, Común",4,Raúl Cuevas Mantecón,4,NA,22 de noviembre de 1977
4,232480,,Pleno,Séptima Época,"Constitucional, Común",5,Arturo Serrano Robles,5,NA,18 de marzo de 1980
5,232480,,Pleno,Séptima Época,"Constitucional, Común",6,Mario G. Rebolledo,6,NA,8 de mayo de 1979
6,232480,,Pleno,Séptima Época,"Constitucional, Común",7,Atanasio González Martínez,7,NA,18 de noviembre de 1980
7,232480,,Pleno,Séptima Época,"Constitucional, Común",8,Fernando Castellanos Tena,8,NA,17 de marzo de 1981
8,232480,,Pleno,Séptima Época,"Constitucional, Común",9,Atanasio González Martínez,9,NA,17 de marzo de 1981
9,232480,,Pleno,Séptima Época,"Constitucional, Común",10,NA,10,NA,16 de febrero de 1982


In [14]:
text_aux

'Tesis Registro digital:  Instancia: Pleno###Séptima Época###Materia(s): Constitucional, Común###Fuente: Semanario Judicial de la Federación. Volumen 157-162, Primera Parte, página 305###Tipo: Jurisprudencia###AUDIENCIA, GARANTIA DE. OBLIGACIONES DEL PODER LEGISLATIVO FRENTE A LOS PARTICULARES.###La Suprema Corte ha resuelto que la garantía de audiencia debe constituir un derecho de los particulares, no sólo frente a las autoridades administrativas y judiciales, sino también frente a la autoridad legislativa, que queda obligada a consignar en sus leyes los procedimientos necesarios para que se oiga a los interesados y se les dé oportunidad de defensa en aquellos casos en que resulten afectados sus derechos. Tal obligación constitucional se circunscribe a señalar el procedimiento aludido; pero no debe ampliarse el criterio hasta el extremo de que los órganos legislativos estén obligados a oír a los posibles afectados por una ley antes de que ésta se expida, ya que resulta imposible sabe

In [15]:
ponente_fecha_aux

[(' en revisión 1501/53. Leonardo Barrera Román y coagraviados. 11 de junio de 1968. Unanimidad de veinte votos. ',
  'Ponente: ',
  '',
  ''),
 (' en revisión 3708/75. José María Escobar Olivas. 5 de marzo de 1977. Unanimidad de dieciséis votos. ',
  'Ponente: ',
  '',
  ''),
 (' en revisión 6163/75. Juan Ramos Russell y otros. 15 de marzo de 1977. Unanimidad de dieciséis votos. ',
  'Ponente: ',
  '',
  ''),
 (' en revisión 5847/76. Eduardo Roberto Cázares G. Cantón y otros. 22 de noviembre de 1977. Unanimidad de dieciocho votos. ',
  'Ponente: ',
  '',
  ''),
 (' en revisión 6408/76. María Fortes de Lamas y otro. 18 de marzo de 1980. Unanimidad de dieciséis votos. ',
  'Ponente: ',
  '',
  ''),
 (' en revisión 2019/78. Máximo Jiménez García. 8 de mayo de 1979. Unanimidad de quince votos. ',
  'Ponente: ',
  '',
  ''),
 (' en revisión 3957/76. Estacionamientos San Francisco, S.A. 18 de noviembre de 1980. Unanimidad de diecinueve votos. ',
  'Ponente: ',
  '',
  ''),
 (' en revisión 7

In [66]:
len(re.findall('La publicación no menciona el nombre del ponente',text_aux))

1

In [67]:
re.findall('La publicación no menciona el nombre del ponente',text_aux)

['La publicación no menciona el nombre del ponente']

In [68]:
[('NA') for i in range(len(re.findall('La publicación no menciona el nombre del ponente',text_aux)))]

['NA']

In [70]:
re.findall(
        "(?:Ponente: Secretario Lic\. |Ponente: |Ponente |Ponente. |Ponentes: |Ponente:\.|Ponente:)([\s\S]*?)(?:.###|. Secretar|.Secretar|,|.\nSecretar|\. secretari|\. Relacionada|\. Disidente|\. Improcedencia|\. Semanario|\. Encargad|\. Competencia|\. Incidente|\. Queja|\. Reclamaci|\. Recurso|\. Juicio|\. Jurisprudencia| Magistrado|\. Vol\wmen|;|\. Nota|\. En| en funciones| en sustitución| y el engrose|Pág\.|Vol|\.Tom|\. V\wase|\. V\wanse|\. Revisión|\. Amparo|\. Ausente|\. Séptima|\. Sexta|\.\"\.)",
        text_aux)

['La publicación no menciona el nombre del ponente',
 'Arturo Martínez Adame',
 'Euquerio Guerrero López',
 'Alfonso López Aparicio',
 'María Cristina Salmorán de Tamayo']

In [23]:
dir_text = "../data_04/Tesis"+tesis_a_revisar+".txt"

file = open(dir_text,'r', encoding='utf8')
text = file.read()
file.close()
    
#Eliminar tabulaciones adicionales
text_aux = text.strip()

#Busca tipo de Tesis
pos_tipo = text_aux.find('Tipo:')
if text_aux[pos_tipo:].find('\n\n') < text_aux[pos_tipo:].find('\n') :
    pos_final_tipo = text_aux[pos_tipo:].find('\n\n')
else:
    pos_final_tipo = text_aux[pos_tipo:].find('\n')-2
len_tipo = len('Tipo:')
tipo = text_aux[pos_tipo+len_tipo:pos_tipo+pos_final_tipo+2].replace('\n',' ').strip()

if tipo == "Jurisprudencia":
        
    if len(re.findall('Ponente',text_aux)) >= 5:
        
        #Datos encabezado
        
        #Posicion en el texto de las variables
        pos_registro_digital = text_aux.find('Registro digital:')
        pos_final_registro_digital = text_aux[pos_registro_digital:].find('\n')

        pos_instancia = text_aux.find('Instancia:')
        pos_final_instancia_aux1 = text_aux[pos_instancia:].find('\n\n')

        lista  = []

        lista.append(text_aux[pos_instancia:].find('Primera Época'))
        lista.append(text_aux[pos_instancia:].find('Segunda Época'))
        lista.append(text_aux[pos_instancia:].find('Tercera Época'))
        lista.append(text_aux[pos_instancia:].find('Cuarta Época'))
        lista.append(text_aux[pos_instancia:].find('Quinta Época'))
        lista.append(text_aux[pos_instancia:].find('Sexta Época'))
        lista.append(text_aux[pos_instancia:].find('Séptima Época'))
        lista.append(text_aux[pos_instancia:].find('Octava Época'))
        lista.append(text_aux[pos_instancia:].find('Novena Época'))
        lista.append(text_aux[pos_instancia:].find('Décima Época'))
        lista.append(text_aux[pos_instancia:].find('Undécima Época'))

        pos_final_instancia_aux2 = min([x for x in lista if x > -1])

        if pos_final_instancia_aux1 < pos_final_instancia_aux2:
            pos_final_instancia = pos_final_instancia_aux1
        else:
            pos_final_instancia = pos_final_instancia_aux2

        pos_epoca_aux1 = text_aux.find('\n\n')

        lista  = []

        lista.append(text_aux.find('Primera Época'))
        lista.append(text_aux.find('Segunda Época'))
        lista.append(text_aux.find('Tercera Época'))
        lista.append(text_aux.find('Cuarta Época'))
        lista.append(text_aux.find('Quinta Época'))
        lista.append(text_aux.find('Sexta Época'))
        lista.append(text_aux.find('Séptima Época'))
        lista.append(text_aux.find('Octava Época'))
        lista.append(text_aux.find('Novena Época'))
        lista.append(text_aux.find('Décima Época'))
        lista.append(text_aux.find('Undécima Época'))

        pos_epoca_aux2 = min([x for x in lista if x > -1])

        if pos_epoca_aux1 < pos_epoca_aux2:
            pos_epoca = pos_epoca_aux1
        else:
            pos_epoca = pos_epoca_aux2

        if pos_epoca_aux1 < pos_epoca_aux2:
            pos_final_epoca = text_aux[pos_epoca+2:].find('\n\n')
        else:
            pos_final_epoca = text_aux[pos_epoca+2:].find('\n')

        pos_materia = text_aux.find('Materia(s):')
        pos_final_materia = text_aux[pos_materia:].find('\n\n')

        pos_tesis = text_aux.find('Tesis:')
        pos_final_tesis = text_aux[pos_tesis:].find('\n\n')

        pos_encabezado = text_aux[pos_tipo:].find('\n\n')

        #Tamano de las variables
        len_registro_digital = len('Registro digital:')
        len_instancia = len('Instancia:')
        if pos_epoca_aux1 < pos_epoca_aux2:
            len_epoca = len('\n\n')
        else:
            len_epoca = 0
        len_materia = len('Materia(s):')
        len_tesis = len('Tesis:')
        len_tipo = len('Tipo:')

        tesis = text_aux[pos_tesis+len_tesis:pos_tesis+pos_final_tesis+2].replace('\n',' ').strip()
        instancia = text_aux[pos_instancia+len_instancia:pos_instancia+pos_final_instancia].replace('\n',' ').strip()
        epoca = text_aux[pos_epoca+len_epoca:pos_epoca+pos_final_epoca+2].replace('\n',' ').strip()
        materia = text_aux[pos_materia+len_materia:pos_materia+pos_final_materia+2].replace('\n',' ').strip()
        
        #Búsqueda de Id
        encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
        id_tesis = [encontrar_id_tesis.group(1)]

        #Limpieza Texto
        tesis_borrar = ''.join(id_tesis)
        text_aux = text_aux.strip()
        text_aux = text_aux.replace('\n\n','###')
        text_aux = text_aux.replace('\n',' ')
        text_aux = text_aux.replace('Ley de Amparo','ley_de_amparo')
        text_aux = text_aux.replace('Competencia Económica','competencia_económica')
        text_aux = text_aux.replace('Fomento a la Competencia','Fomento a la competencia')
        text_aux = text_aux.replace('del año','de')
        text_aux = text_aux.replace('Presidente Ponente','Presidente-ponente')
        text_aux = text_aux.replace('autorizados por el Secretario','autorizados por el secr.')
        text_aux = text_aux.replace('Ponente: La publicación no menciona el nombre del ponente.','La publicación no menciona el nombre del ponente.')
        text_aux = text_aux.replace(' en sesión de 5 de marzo de 1996','')
        text_aux = text_aux.replace('29 se septiembre de 1993','29 de septiembre de 1993')
        text_aux = text_aux.replace('12 se agosto de 1976','12 de agosto de 1976')
        text_aux = text_aux.replace('10 de 1988','10 de FALTAMES de 1988')
        text_aux = text_aux.replace('* de abril de 1969','99 de abril de 1969')
        text_aux = text_aux.replace('7 de agosto de 1984. 7 de agosto de 1984.','7 de agosto de 1984.')
        text_aux = text_aux.replace('4 de julio de 975','4 de julio de 1975')
        text_aux = text_aux.replace(r'1o.','01')
        text_aux = text_aux.replace(r'1°','01')
        text_aux = text_aux.replace(r'1º','01')
        text_aux = text_aux.replace('Pág. 1 de 1 ','')
        text_aux = text_aux.replace('Pág. 1 de 2 ','')
        text_aux = text_aux.replace('Pág. 2 de 2 ','')
        text_aux = text_aux.replace('Pág. 1 de 3 ','')
        text_aux = text_aux.replace('Pág. 2 de 3 ','')
        text_aux = text_aux.replace('Pág. 3 de 3 ','')
        text_aux = text_aux.replace('Pág. 1 de 4 ','')
        text_aux = text_aux.replace('Pág. 2 de 4 ','')
        text_aux = text_aux.replace('Pág. 3 de 4 ','')
        text_aux = text_aux.replace('Pág. 4 de 4 ','')
        text_aux = text_aux.replace('Pág. 1 de 5 ','')
        text_aux = text_aux.replace('Pág. 2 de 5 ','')
        text_aux = text_aux.replace('Pág. 3 de 5 ','')
        text_aux = text_aux.replace('Pág. 4 de 5 ','')
        text_aux = text_aux.replace('Pág. 5 de 5 ','')
        text_aux = text_aux.replace('Pág. 1 de 6 ','')
        text_aux = text_aux.replace('Pág. 2 de 6 ','')
        text_aux = text_aux.replace('Pág. 3 de 6 ','')
        text_aux = text_aux.replace('Pág. 4 de 6 ','')
        text_aux = text_aux.replace('Pág. 5 de 6 ','')
        text_aux = text_aux.replace('Pág. 6 de 6 ','')
        text_aux = text_aux.replace('http://sjf2.scjn.gob.mx/detalle/tesis/','')
        text_aux = text_aux.replace(tesis_borrar,'')
        text_aux = text_aux.replace('  Fecha de impresión','')
        text_aux = text_aux.replace('Fecha de impresión','')
        text_aux = text_aux.replace(' 06/07/2021','')
        text_aux = text_aux.replace(' 07/07/2021','')
        text_aux = text_aux.replace(' 08/07/2021','')
        text_aux = text_aux.replace(' 09/07/2021','')
        text_aux = text_aux.replace(' 10/07/2021','')
        text_aux = text_aux.replace('  \x0c','')
        text_aux = text_aux.replace('######\x0c','')
        text_aux = text_aux.replace('Semanario Judicial de la Federación ','')
        text_aux = re.sub(r'Se formó jurisprudencia aunque no.*',"",text_aux)
        text_aux = text_aux.replace(r'(DA. 903/88. Impulsora Industrial de Ingeniería, S. A. de C. V. 15 de junio de 1988. Unanimidad de votos. Ponente: Genaro David Góngora Pimentel. Secretaria: Adriana Leticia Campuzano Gallegos)','')
        #text_aux = text_aux.replace('######','###')

        #Búsqueda de párrafo donde se mencione ponente con fecha
        #ponente_fecha_aux = re.findall(r'(?:Amparo|Revisión|Contradicción|Queja|Facultad|Incidente|Reclamación|Inconformidad|Competencia|Recurso de|Impedimento|Acción de inconstitucionalidad)([\s\S]*?)(Ponente: |Ponente )([\s\S]*?)(Secretari\w: |Secretari\ws:|Secretari\w. )([\s\S]*?)(?:\.?)', text_aux)
        ponente_fecha_aux = re.findall(r'(?:Amparo|Revisión|Contradicción|Queja|Facultad|Incidente|Reclamación|Inconformidad|Competencia|Recurso de|Impedimento|Acción de inconstitucionalidad|Varios |Controversia constitucional|Reconocimiento de inocencia|Improcedencia|Aclaración de sentencia en la contra|Repetición|Inejecuci|Expediente vario|Solicitud|Acumulaci\wn|Trámite|Consulta|Recurso en|Juicio|Reposici\wn|Incompetencia|Incidente|Vol\wmen \d{1,2}, p\wgina|Indulto necesario)([\s\S]*?)(Ponente: |Ponente |Ponente\.|Ponente:\.|La publicación no menciona el nombre del ponente)([\s\S]*?)(?:Secretari\w: |Secretari\ws:|Secretari\w. )?([\s\S]*?)(?:\.?)', text_aux)
        ponente_fecha_texto = ''.join([''.join(j) for j in ponente_fecha_aux])
        
        #Búsqueda de Ponente
        #x0_ponente = re.findall("Ponente:([\s\S]*?)(?:.###|. Secretari|.Secretari|,|.\nSecretari|;|. Nota:|Pág.|Vol|.Tom|. Véase|. Amparo|. Séptima Época|. Sexta Época)",ponente_fecha_texto) 
        x0_ponente = re.findall("(?:Ponente: Secretario Lic\. |Ponente: |Ponente |Ponente. |Ponentes: |Ponente:\.|Ponente:)([\s\S]*?)(?:.###|. Secretar|.Secretar|,|.\nSecretar|\. secretari|\. Relacionada|\. Disidente|\. Improcedencia|\. Semanario|\. Encargad|\. Competencia|\. Incidente|\. Queja|\. Reclamaci|\. Recurso|\. Juicio|\. Jurisprudencia| Magistrado|\. Tomo|\. Vol\wmen|;|\. Nota|\. En| en funciones| en sustitución| y el engrose|Pág\.|Vol|\.Tom|\. V\wase|\. V\wanse|\. Revisión|\. Amparo|\. Ausente|\. Séptima|\. Sexta|\.\"\.)",text_aux) 
        x1_ponente = re.findall("publicación ([\s\S]*?) del ponente",text_aux )
        x2_ponente = re.findall("Relator: ([\s\S]*?)\.",text_aux ) #re.findall(r"Relator",text_aux)
        x3_ponente = 1
        ponentes = funcion_auxiliar_ponentes(x0_ponente,x1_ponente,x2_ponente,x3_ponente)
        
        if len(re.findall('La publicación no menciona el nombre del ponente',text_aux)) != 0: 
            NA_ponentes = [('NA') for i in range(len(re.findall('La publicación no menciona el nombre del ponente',text_aux)))]
            ponentes.extend(NA_ponentes)
        if len(x2_ponente) != 0: 
            ponentes.extend(x2_ponente)

        #Búsqueda de Fecha
        #fecha_ponente = re.findall(r'\d{1,2} de [a-z]*? de \d{4}', ponente_fecha_texto)
        #fecha = re.findall('(\d{1,2} de [a-z]*?( de)*? \d{4})',ponente_fecha_texto)
        fecha = re.findall('(\d{1,2}( de|de)*? [a-zA-Z]{4,10}( de| del)*? \d{4})',ponente_fecha_texto)
        fecha_aux = pd.DataFrame(fecha,columns=['fecha','del1','del2']).drop(columns=['del1','del2'])
        fecha = fecha_aux['fecha'].values.tolist()
        if len(re.findall('La publicación no menciona la fecha de resolución',text_aux)) != 0: 
            NA_fecha = [('NA') for i in range(len(re.findall('La publicación no menciona la fecha de resolución',text_aux)))]
            fecha.extend(NA_fecha)
                
        #Búsqueda de Secretario
        #x0_secretario = re.findall("(?:Secretari\w: |Secretari\ws: |Secretari\w. )([\s\S]*?)(\.| Amparo)",text_aux)
        #x0_aux = pd.DataFrame(x0_secretario,columns=['secretario','delete']).drop(columns=['delete'])
        #x0_secretario = x0_aux['secretario'].values.tolist()
        #x1_secretario = re.findall("(?:La publicación |La publicacion)([\s\S]*?) (?:del ponente|el ponente)",text_aux)
        ##x10_secretario = 10
        ##secretarios = funcion_auxiliar_secretarios(x0_secretario,x1_secretario) #,x10_secretario
        #secretarios = x0_secretario
        
        x0_secretario = re.findall("(?:Secretar\w\w: |Secretari\ws: |Secretari\w\. |secretari\w: )([\s\S]*?)(?:\.###|\. Amparo|\. Facultad| Amparo directo|\. Inconformidad|\. Acción|\. Varios|\. Controversia|\. Reconocimiento|\. Expediente vario|\. Consulta|\. Reposici\wn|\. \(|\. La |\. Incompetencia|\. Vol\wmenes|se encargo|Quinta Epoca:|\. Hay cosa|\. Acumulación|Impedimento|Antecedente:|Revisión|Expediente|Los señores|Jurisprudencia|demás que las|XIV.|las demás que|fallado|no superan|Aplicada|De conformidad|contra el|para el|Lo resolvió|Secretarrio:|Presidente|Secretario encargado|ha estimado|Disidente.|Veáse|\. Impedido:|Vease:|Por |Sexta Epoca|Integró|Para|Tribunal|\. Ausente|votos|_________________|Texto|Secretarios|Reproduce|Véanse:|Ponente:|EL|Tomo|Resuelto|Ausentes|Que |Varios|Con |\. Aclaración|Tomando|\. Juicio|Relator|\. Ministro|Incidente|/.|Texto aprobado|Relacionada|Séptima|Acción|Facultad|Controversia|\. Solicitud|Sostiene|La |\. Inejecución|Octava|En |Este |Se |\. Repetición|Reitera|\. El |Contradicción|La presente|El Tribunal|Tesis|La Primera|Sostienen|Informe|Véase|Esta|Importa|Disidentes:|Voto|Ausente:|Secretario:|###|Inconformidad|\. Conflicto|Nota|Criterios|Competencia|Precedente|\. Recurso| Recurso de|Secretaria|;|\. Nota\:|\. Notas\:|Nota\:|Pág\.|Vol|\. Volumen|\. Volúme|\.Tom|\. Véase|\. \. Amparo|\. Amparo di|\.   Amparo dir|\. Amparo in|Amparo en|\. Undécima Época|\. Décima Época|\. Novena Época|\. Octava Época|\. Séptima Época|\. Sexta Época|\. Quinta Época|\. Cuarta Época|\. Tercera Época|\. Segunda Época|\. Primera Época|\. Undécima Epoca|\. Décima Epoca|\. Novena Epoca|\. Octava Epoca|\. Séptima Epoca|\. Sexta Epoca|\. Quinta Epoca|\. Cuarta Epoca|\. Tercera Epoca|\. Segunda Epoca|\. Primera Epoca|\. Improcedencia civil|Sostiene la misma tesis:|\. Revisión fiscal|\. Unanimidad|\. Queja|Queja|\. Competencia|\. Reclamación|\.   Reclamación /.|\.  de enero de|\.  de febrero de|\.  de marzo de|\.  de abril de|\.  de mayo de|\.  de junio de|\.  de julio de|\.  de agosto de|\.  de septiembre de|\.  de octubre de|\.  de noviembre de|\.  de diciembre de|\. Disidente:|\. Precedentes|\. Improcedencia|\. Impedimento|\. Excusa|\. Incidente|\. Encargad|\. Revisión|\. Tomo|por licencia concedida|en funciones de Magistrado por Ministerio de Ley|\. Engrose: |\. Criterios|\. Tesis|\.   Tesis|\. El Tribunal|\. Contradicción de tesis|\. Texto|\. NOTA|\. Recurso de|\.   El Tribunal|\. Véanse|\.   Revisión|.   Notas|\. Semanario|\. Secretario)", text_aux)                                
        x1_secretario = re.findall("(?:La publicación |La publicacion)([\s\S]*?) (?:del ponente|el ponente)",text_aux)
        x10_secretario = 1
        secretarios = funcion_auxiliar_secretarios(x0_secretario,x1_secretario,x10_secretario)
        if len(secretarios) != len(ponentes):
            NA_sec = [('NA') for i in range(len(ponentes)-len(secretarios))]
            secretarios.extend(NA_sec)
    
        #impresión debug
        print(id_tesis)
        print(fecha)
        print('Ponentes:',ponentes)
        print('Secretarios:',secretarios)

        #Vectores para crear tabla
        vector_id_tesis = len(ponentes)*id_tesis
        vector_numero = [(i+1) for i in range(len(ponentes))]
        vector_tesis = [(tesis) for i in range(len(ponentes))]
        vector_instancia = [(instancia) for i in range(len(ponentes))]
        vector_epoca = [(epoca) for i in range(len(ponentes))]
        vector_materia = [(materia) for i in range(len(ponentes))]

        tabla_iteracion = pd.DataFrame({'id_tesis':vector_id_tesis,'tesis':vector_tesis,'instancia':vector_instancia,'epoca':vector_epoca,'materia':vector_materia,'#_ponente':vector_numero,'ponente':ponentes,'#_secretario':vector_numero, 'secretario':secretarios,'fecha':fecha})
        #tabla_tesis_jurisprudencia = tabla_tesis_jurisprudencia.append(tabla_iteracion)
        
        print(instancia)
        print(epoca)
        print(materia)
        print(tesis)

['232480']
['11 de junio de 1968', '5 de marzo de 1977', '15 de marzo de 1977', '22 de noviembre de 1977', '18 de marzo de 1980', '8 de mayo de 1979', '18 de noviembre de 1980', '17 de marzo de 1981', '17 de marzo de 1981', '16 de febrero de 1982']
Ponentes: ['Ernesto Solís López', 'Raúl Cuevas Mantecón', 'J. Ramón Palacios Vargas', 'Raúl Cuevas Mantecón', 'Arturo Serrano Robles', 'Mario G. Rebolledo', 'Atanasio González Martínez', 'Fernando Castellanos Tena', 'Atanasio González Martínez', 'NA']
Secretarios: ['NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA']
Pleno
Séptima Época
Constitucional, Común



## Leyendo todas las tesis en bloques de 20,000

In [3]:
tabla_tesis_jurisprudencia_01 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_02 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_03 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_04 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_05 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_06 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_07 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_08 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_09 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_10 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_11 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_12 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])
tabla_tesis_jurisprudencia_13 = pd.DataFrame(columns=['id_tesis','tesis','instancia','epoca','materia','#_ponente','ponente','#_secretario','secretario','fecha'])

In [4]:
ruta_txt = '../data/*.txt'  #Total archivos: 260,303

**0 a 20,000**

In [26]:
ruta_txt = '../data_01/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #len(range(0,20000))
n_juris_reiter_01 = 0

for i in range(n):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_01 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_01 = tabla_tesis_jurisprudencia_01.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_01, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_01)


Van 1000 iteraciones de 20000 en 38.61 segundos
Hay 187 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 84.11 segundos
Hay 313 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 133.80 segundos
Hay 412 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 183.73 segundos
Hay 546 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 234.66 segundos
Hay 665 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 284.45 segundos
Hay 787 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 334.04 segundos
Hay 902 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 382.66 segundos
Hay 1013 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 431.01 segundos

In [27]:
tabla_tesis_jurisprudencia_01.describe()

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
count,12095,12095,12095,12095,12095,12095,12095,12095,12095,12095
unique,2419,2412,4,2,29,5,714,5,2212,2196
top,159807,VI.3o.A. J/61,Tribunales Colegiados de Circuito,Novena Época,Administrativa,1,José Ramón Cossío Díaz,1,"Georgina Laso de la Vega Romero, Sofía Verónic...",19 de junio de 2008
freq,5,10,7300,11090,2550,2419,670,2419,449,460


In [28]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_01).to_csv('tabla_tesis_jurisprudencia_01.csv', index=False, encoding='utf-8-sig')

**20,000 a 40,000**

In [29]:
ruta_txt = '../data_02/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(20000,40000))
n_juris_reiter_02 = 0

for i in range(n): #for i in range(20000,40000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_02 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_02 = tabla_tesis_jurisprudencia_02.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_02, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_02)


Van 1000 iteraciones de 20000 en 46.06 segundos
Hay 106 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 93.20 segundos
Hay 193 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 140.56 segundos
Hay 285 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 188.36 segundos
Hay 394 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 235.76 segundos
Hay 505 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 283.28 segundos
Hay 604 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 330.91 segundos
Hay 701 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 378.05 segundos
Hay 810 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 425.82 segundos


In [30]:
tabla_tesis_jurisprudencia_02.describe()

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
count,12475,12475,12475,12475,12475,12475,12475,12475,12475,12475
unique,2494,2492,5,1,33,10,754,10,2346,2531
top,201624,V.3o. J/2,Tribunales Colegiados de Circuito,Novena Época,Común,1,Mariano Azuela Güitrón,1,Lourdes Ferrer Mac Gregor Poisot,11 de septiembre de 2000
freq,10,10,8610,12475,2520,2494,610,2494,202,92


In [31]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_02).to_csv('tabla_tesis_jurisprudencia_02.csv', index=False, encoding='utf-8-sig')

**40,000 a 60,000**

In [32]:
ruta_txt = '../data_03/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(40000,60000))
n_juris_reiter_03 = 0

for i in range(n): #for i in range(40000,60000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_03 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)
                
                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_03 = tabla_tesis_jurisprudencia_03.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_03, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_03)


Van 1000 iteraciones de 20000 en 37.38 segundos
Hay 135 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 79.11 segundos
Hay 277 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 124.18 segundos
Hay 396 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 172.57 segundos
Hay 590 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 220.70 segundos
Hay 742 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 268.94 segundos
Hay 833 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 317.57 segundos
Hay 915 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 365.53 segundos
Hay 1044 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 413.30 segundos

In [33]:
tabla_tesis_jurisprudencia_03.describe()

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
count,11430,11430,11430,11430,11430,11430,11430,11430,11430,11430
unique,2286,2172,6,2,25,5,408,5,1424,1874
top,202293,,Tribunales Colegiados de Circuito,Octava Época,Común,1,Gustavo Calvillo Rangel,1,Jorge Alberto González Alvarez,15 de mayo de 1991
freq,5,30,9425,9415,3205,2286,662,2286,275,75


In [34]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_03).to_csv('tabla_tesis_jurisprudencia_03.csv', index=False, encoding='utf-8-sig')

**60,000 a 80,000**

In [35]:
ruta_txt = '../data_04/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(60000,80000))
n_juris_reiter_04 = 0

for i in range(n): #for i in range(60000,80000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_04 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_04 = tabla_tesis_jurisprudencia_04.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_04, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_04)


Van 1000 iteraciones de 20000 en 39.34 segundos
Hay 135 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 83.16 segundos
Hay 150 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 130.36 segundos
Hay 279 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 178.94 segundos
Hay 385 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 227.12 segundos
Hay 417 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 275.80 segundos
Hay 525 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 323.81 segundos
Hay 607 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 370.83 segundos
Hay 608 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 417.93 segundos


In [36]:
tabla_tesis_jurisprudencia_04.describe()

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
count,6394,6394,6394,6394,6394,6394,6394,6394,6394,6394
unique,1272,701,6,2,28,10,330,10,868,2322
top,232480,,Tribunales Colegiados de Circuito,Octava Época,Administrativa,1,Carlos del Río Rodríguez,1,NA,31 de agosto de 1976
freq,10,2886,3513,3513,2136,1272,482,1272,2256,25


In [37]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_04).to_csv('tabla_tesis_jurisprudencia_04.csv', index=False, encoding='utf-8-sig')

**80,000 a 100,000**

In [4]:
ruta_txt = '../data_05/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(80000,100000))
n_juris_reiter_05 = 0

for i in range(n): #for i in range(80000,100000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()
        
        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_05 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_05 = tabla_tesis_jurisprudencia_05.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_05, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_05)


Van 1000 iteraciones de 20000 en 36.48 segundos
Hay 223 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 78.13 segundos
Hay 247 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 122.54 segundos
Hay 259 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 168.72 segundos
Hay 270 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 214.29 segundos
Hay 278 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 258.67 segundos
Hay 289 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 302.92 segundos
Hay 308 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 346.47 segundos
Hay 318 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 389.63 segundos


In [5]:
tabla_tesis_jurisprudencia_05.describe()

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
count,2146,2146,2146,2146,2146,2146,2146,2146,2146,2146
unique,422,1,5,2,16,8,164,8,232,1334
top,242893,,Cuarta Sala,Séptima Época,Laboral,1,María Cristina Salmorán de Tamayo,1,NA,29 de octubre de 1970
freq,8,2146,1320,2066,1306,422,412,422,1233,11


In [6]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_05).to_csv('tabla_tesis_jurisprudencia_05.csv', index=False, encoding='utf-8-sig')

**100,000 a 120,000**

In [8]:
ruta_txt = '../data_06/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(100000,120000))
n_juris_reiter_06 = 0

for i in range(n): #for i in range(100000,120000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_06 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_06 = tabla_tesis_jurisprudencia_06.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_06, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_06)


Van 1000 iteraciones de 20000 en 40.29 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 85.87 segundos
Hay 5 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 134.27 segundos
Hay 11 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 182.86 segundos
Hay 12 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 231.18 segundos
Hay 14 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 279.52 segundos
Hay 15 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 327.57 segundos
Hay 18 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 375.16 segundos
Hay 19 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 423.81 segundos
Hay 24 tes

In [9]:
tabla_tesis_jurisprudencia_06.describe()

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
count,135,135,135,135,135,135,135,135,135,135
unique,27,1,4,1,6,5,27,5,1,120
top,264103,,Segunda Sala,Sexta Época,Administrativa,1,Felipe Tena Ramírez,1,NA,29 de abril de 1958
freq,5,135,70,135,55,27,18,27,135,3


In [10]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_06).to_csv('tabla_tesis_jurisprudencia_06.csv', index=False, encoding='utf-8-sig')

**120,000 a 140,000**

In [11]:
ruta_txt = '../data_07/*.txt'

t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(120000,140000))
n_juris_reiter_07 = 0

for i in range(n): #for i in range(120000,140000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_07 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_07 = tabla_tesis_jurisprudencia_07.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_07, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_07)



Van 1000 iteraciones de 20000 en 42.82 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 90.49 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 140.40 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 190.39 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 240.44 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 290.78 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 340.41 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 389.41 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 438.86 segundos
Hay 0 tesis de J

In [12]:
tabla_tesis_jurisprudencia_07.describe()

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
count,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_07).to_csv('tabla_tesis_jurisprudencia_07.csv', index=False, encoding='utf-8-sig')

**140,000 a 160,000**

In [14]:
ruta_txt = '../data_08/*.txt'
    
t0 = time.time()
n = len(glob.glob(ruta_txt)) #n = len(range(140000,160000))
n_juris_reiter_08 = 0

for i in range(n): #for i in range(140000,160000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_08 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_08 = tabla_tesis_jurisprudencia_08.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_08, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_08)



Van 1000 iteraciones de 20000 en 42.46 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 1000 tesis leídas

Van 2000 iteraciones de 20000 en 90.24 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 2000 tesis leídas

Van 3000 iteraciones de 20000 en 140.09 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 3000 tesis leídas

Van 4000 iteraciones de 20000 en 189.83 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 4000 tesis leídas

Van 5000 iteraciones de 20000 en 239.41 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 5000 tesis leídas

Van 6000 iteraciones de 20000 en 289.12 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 6000 tesis leídas

Van 7000 iteraciones de 20000 en 337.86 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 7000 tesis leídas

Van 8000 iteraciones de 20000 en 386.41 segundos
Hay 0 tesis de Jurisprudencia por Reiteración en 8000 tesis leídas

Van 9000 iteraciones de 20000 en 435.56 segundos
Hay 0 tesis de J

In [15]:
tabla_tesis_jurisprudencia_08.describe()

,id_tesis,tesis,instancia,epoca,materia,#_ponente,ponente,#_secretario,secretario,fecha
count,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_08).to_csv('tabla_tesis_jurisprudencia_08.csv', index=False, encoding='utf-8-sig')

**160,000 a 180,000**

In [ ]:
t0 = time.time()
n = len(range(160000,180000))
n_juris_reiter_09 = 0

for i in range(160000,180000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_09 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_09 = tabla_tesis_jurisprudencia_09.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_09, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_09)


In [ ]:
tabla_tesis_jurisprudencia_09.describe()

In [ ]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_09).to_csv('tabla_tesis_jurisprudencia_09.csv', index=False, encoding='utf-8-sig')

**180,000 a 200,000**

In [ ]:
t0 = time.time()
n = len(range(180000,200000))
n_juris_reiter_10 = 0

for i in range(180000,200000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_10 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_10 = tabla_tesis_jurisprudencia_10.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_10, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_10)


In [ ]:
tabla_tesis_jurisprudencia_10.describe()

In [ ]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_10).to_csv('tabla_tesis_jurisprudencia_10.csv', index=False, encoding='utf-8-sig')

**200,000 a 220,000**

In [ ]:
t0 = time.time()
n = len(range(200000,220000))
n_juris_reiter_11 = 0

for i in range(200000,220000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_11 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_11 = tabla_tesis_jurisprudencia_11.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_11, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_11)


In [ ]:
tabla_tesis_jurisprudencia_11.describe()

In [ ]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_11).to_csv('tabla_tesis_jurisprudencia_11.csv', index=False, encoding='utf-8-sig')

**220,000 a 240,000**

In [ ]:
t0 = time.time()
n = len(range(220000,240000))
n_juris_reiter_12 = 0

for i in range(220000,240000):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_12 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_12 = tabla_tesis_jurisprudencia_12.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_12, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_12)

In [ ]:
tabla_tesis_jurisprudencia_12.describe()

In [ ]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_12).to_csv('tabla_tesis_jurisprudencia_12.csv', index=False, encoding='utf-8-sig')

**240,000 a 260,303**

In [ ]:
t0 = time.time()
n = len(range(240000,260303))
n_juris_reiter_13 = 0

for i in range(240000,260303):
    
    try:

        file = open(glob.glob(ruta_txt)[i],'r', encoding='utf8')
        text = file.read()
        file.close()

        tipo = busca_tipo_tesis(text)

        if tipo == "Jurisprudencia":

            if len(re.findall('Ponente',text)) >= 5:
                
                n_juris_reiter_13 += 1
                
                #Datos encabezado
                datos_encabezado = extraccion_encabezado(text)
                tesis = datos_encabezado[0]
                instancia = datos_encabezado[1]
                epoca = datos_encabezado[2]
                materia = datos_encabezado[3]

                #Búsqueda de Id
                encontrar_id_tesis = re.search(r'Registro digital: (\d+)',text)
                id_tesis = [encontrar_id_tesis.group(1)]

                #Limpieza Texto
                text_aux = limpieza_texto(text,id_tesis)

                #Búsquedas de Ponentes, Secretarios y Fechas
                ponentes = busqueda_ponentes(text_aux)
                secretarios = busqueda_secretarios(text_aux,ponentes)
                fecha = busqueda_fecha(text_aux)

                tabla_iteracion = crea_tabla(id_tesis,tesis,instancia,epoca,materia,ponentes,secretarios,fecha)
                tabla_tesis_jurisprudencia_13 = tabla_tesis_jurisprudencia_13.append(tabla_iteracion)

        # Imprimir cada 1,000 iteraciones
        if i%1000 == 0:
            if i!= 0:
                ti = time.time()
                print("\nVan {ii} iteraciones de {nn} en {tt:.2f} segundos".format(ii=i, nn=n, tt=ti-t0))
                print("Hay {jj} tesis de Jurisprudencia por Reiteración en {ii} tesis leídas".format(jj = n_juris_reiter_13, ii = i)) 
    
    except ValueError:
        print("\nAll arrays must be of the same length")
        print(id_tesis)
        if len(fecha) != 5:
            print(fecha)
        if len(ponentes) != 5:
            print('Ponentes:',ponentes)
        if len(secretarios) != 5:
            print('Secretarios:',secretarios)
    
tf = time.time()
print("\nTiempo total: {tiempo:.2f} segundos".format(tiempo=tf-t0))
print("Total tesis leídas:", n)
print("Total tesis de Jurisprudencia por Reiteración:", n_juris_reiter_13)


In [ ]:
tabla_tesis_jurisprudencia_13.describe()

In [ ]:
#Convierte dataframe a .csv

pd.DataFrame(tabla_tesis_jurisprudencia_13).to_csv('tabla_tesis_jurisprudencia_13.csv', index=False, encoding='utf-8-sig')

In [ ]:
n_total_juris_reiter = n_juris_reiter_01+n_juris_reiter_02+n_juris_reiter_03+n_juris_reiter_04+n_juris_reiter_05+n_juris_reiter_06+n_juris_reiter_07+n_juris_reiter_08+n_juris_reiter_09+n_juris_reiter_10+n_juris_reiter_11+n_juris_reiter_12+n_juris_reiter_13
print("Gran total tesis de Jurisprudencia por Reiteración:", n_total_juris_reiter)